In [1]:
import pandas as pd
import numpy as np

import os
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"


from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
df = pd.read_csv("C:/Users/dutta/Downloads/Lukas_sentinel_project/notebooks/lucas2018_DE_phase3_clean_new.csv")

In [3]:
df.shape
df.head()

,POINT_ID,crop_class,NUTS2,month,NDVI,EVI
0,20180301T103019_20180301T103013_T31UGT_0000000...,barley,DEA3,3,0.543991,1.161269
1,20180301T103019_20180301T103013_T31UGT_0000000...,barley,DEA3,3,0.632279,1.620699
2,20180301T103019_20180301T103013_T31UGT_0000000...,barley,DEA3,3,0.331896,0.658480
3,20180301T103019_20180301T103013_T31UGT_0000000...,barley,DEA3,3,0.312444,0.588335
4,20180301T103019_20180301T103013_T31UGT_0000000...,barley,DEA3,3,0.381513,0.852262


In [ ]:
df.shape

In [ ]:
#keeping only the columns we need

In [ ]:
df.head()
df.shape
df.columns



In [ ]:
#Check month values

df["month"].unique()


In [ ]:
df["month"] = df["month"].astype(int)


In [ ]:
df["month"].unique()

In [ ]:
X = df[["NDVI", "EVI", "month"]].copy()
y = df["crop_class"]

In [ ]:
X[["NDVI", "EVI"]] = X[["NDVI", "EVI"]].clip(-1, 1)


In [ ]:
X = X.astype({
    "NDVI": "float32",
    "EVI": "float32",
    "month": "int8"
})


In [ ]:
#Train/test split

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=42,
    stratify=y
)


In [ ]:
#Train Random Forest


from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=300,
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train, y_train)


In [ ]:
#Accuracy assessment

from sklearn.metrics import classification_report

y_pred = rf.predict(X_test)
print(classification_report(y_test, y_pred))


In [ ]:
#“Using single-date Sentinel-2 vegetation indices and survey month, the Random Forest classifier achieved an overall accuracy of 31%, with substantially higher performance for major arable crops such as maize and wheat. The reduced accuracy for minor crops highlights the limitations of single-date EO observations and motivates the integration of multi-sensor and multi-temporal information.”

In [ ]:
#Use class-weighted Random Forest to help minority crops, this will reduce maize dominance

rf_bal = RandomForestClassifier(
    n_estimators=300,
    random_state=42,
    n_jobs=-1,
    class_weight="balanced"
)

rf_bal.fit(X_train, y_train)

from sklearn.metrics import classification_report
print(classification_report(y_test, rf_bal.predict(X_test)))
